In [ ]:
import os
import subprocess
import rastervision
import rasterio

# os.environ['GDAL_DATA'] = check_output('pip show rasterio | grep Location | awk \'{print $NF"/rasterio/gdal_data/"}\'', shell=True).decode().strip()
# os.environ['AWS_NO_SIGN_REQUEST'] = 'YES'


# Find the location of the 'rasterio' package using pip
try:
    result = subprocess.check_output('pip show rasterio', shell=True).decode('utf-8')

    # Extract the location of the 'rasterio' package
    for line in result.splitlines():
        if line.startswith('Location:'):
            rasterio_location = line.split(':')[-1].strip()
            break
    
    # Set GDAL_DATA environment variable
    gdal_data_path = os.path.join(rasterio_location, 'rasterio', 'gdal_data')
    os.environ['GDAL_DATA'] = gdal_data_path
    print(f"GDAL_DATA has been set to: {gdal_data_path}")

except subprocess.CalledProcessError as e:
    print("Error finding the location of rasterio:", e)

# Set AWS_NO_SIGN_REQUEST environment variable
os.environ['AWS_NO_SIGN_REQUEST'] = 'YES'
print("AWS_NO_SIGN_REQUEST has been set to: YES")


In [ ]:
import gdown
import glob

from rastervision.core.data import RasterioSource, MinMaxTransformer

from rastervision.core.data import (
    ClassConfig, GeoJSONVectorSource, RasterioCRSTransformer,
    RasterizedSource, ClassInferenceTransformer)

from rastervision.core.data import SemanticSegmentationLabelSource

from rastervision.core.data.utils.geojson import get_polygons_from_uris
from shapely.geometry import Polygon

from rastervision.pytorch_learner import (
    SemanticSegmentationRandomWindowGeoDataset, SemanticSegmentationSlidingWindowGeoDataset, SemanticSegmentationVisualizer)

import albumentations as A

import torch
from torch.utils.data import ConcatDataset

from rastervision.pytorch_learner import SemanticSegmentationGeoDataConfig
from rastervision.pytorch_learner import SolverConfig
from rastervision.pytorch_learner import SemanticSegmentationLearnerConfig
from rastervision.pytorch_learner import SemanticSegmentationLearner


In [ ]:
!pip install raster-vision/rastervision_core

ERROR: Invalid requirement: 'raster-vision/rastervision_core': Expected end or semicolon (after name and no valid version specifier)
    raster-vision/rastervision_core
                 ^


In [ ]:
images_val
annotations_val
images_train
annotations_train

In [ ]:
# Aquí definimos las clases (tipos de objeto) presentes en nuestras anotaciones
# en este caso tenemos sólo una, que indicamos como "basural" .
# hay un clase adicional, implícita, que es "background" -el fondo, todo lo que
# no corresponde a objetos de intéres

class_config = ClassConfig(
    names=['background', 'vivienda precaria'],
    colors=['lightgray', 'darkred'],
    null_class='background')


# el tamaño en píxeles de los recortes cuadrados
window_size = 480

# Aquí definimos algunas transformaciones a realizar a los recorte del dataset
# de entrenamiento: cambiar al azar la saturación, el brillo, rotarlos, ocultar
# algunos pixeles. Todo esto sirve para entrenar un algoritmo de detección
# más robusto a diferencias que puedan tener las futuras imágenes a las que
# se aplique
data_augmentation_transform = A.Compose([
    A.Flip(),
    A.ShiftScaleRotate(),
    A.OneOf([
        A.HueSaturationValue(hue_shift_limit=10),
        A.RandomBrightnessContrast(),
        A.RandomGamma(),
    ]),
    A.CoarseDropout(max_height=int(window_size/6), max_width=int(window_size/6), max_holes=4)
])


In [ ]:
val_ds = SemanticSegmentationSlidingWindowGeoDataset.from_uris(
    class_config=class_config,
    aoi_uri=AoI_file_dest,
    # aoi_uri=annotations_file_dest,
    image_uri=images_val,
    label_vector_uri=annotations_val,
    label_vector_default_class_id=class_config.get_class_id('vivienda precaria'),
    image_raster_source_kw=dict(allow_streaming=True, raster_transformers=[MinMaxTransformer()]),
    size=window_size,
    stride=window_size,
    transform=A.Resize(window_size, window_size))

In [ ]:
vis = SemanticSegmentationVisualizer(
    class_names=class_config.names, class_colors=class_config.colors)

x, y = val_ds[1]

vis.plot_batch(x.unsqueeze(0), y.unsqueeze(0), show=True)

In [ ]:
# Aquí definimos cuantos recortes sobre la imagen tomaremos al azar
# idealmente, varias veces la cantidad de recortes en el dataset de validación
# (algo así como sample_size = len(val_ds) * 5
# pero como estos datos son "pesados" vamos a limitarnos a tomar una muestra de la misma cantidad
sample_size = len(val_ds)*5

train_ds = SemanticSegmentationRandomWindowGeoDataset.from_uris(
    class_config=class_config,
    # aoi_uri=AoI_file_dest,
    image_uri=images_train,
    label_vector_uri=annotations_train,
    label_vector_default_class_id=class_config.get_class_id('asentamiento informal'),
    image_raster_source_kw=dict(allow_streaming=True, raster_transformers=[MinMaxTransformer()]),
    # window sizes will randomly vary from 100x100 to 300x300
    #size_lims=(100, 300),
    # fixed window size
    size_lims=(window_size, window_size+1),
    # resize chips before returning
    out_size=window_size,
    # allow windows to overflow the extent by 100 pixels
    padding=100,
    max_windows=sample_size, # pero como estos datos son "pesados" vamos a limitarnos a tomar una muestra de la misma cantidad
    transform=data_augmentation_transform
)

In [ ]:
x, y = vis.get_batch(train_ds, 8)

vis.plot_batch(x, y, show=True)

In [ ]:
import torch

model = torch.hub.load(
    'AdeelH/pytorch-fpn:0.3',
    'make_fpn_resnet',
    name='resnet18',
    fpn_type='panoptic',
    num_classes=len(class_config),
    fpn_channels=128,
    in_channels=3,
    out_size=(window_size, window_size),
    pretrained=True)


In [ ]:
learner.train(epochs=n_epochs)

In [ ]:
learner.save_model_bundle()